# Object Segmenation on Azure Stack Hub Clusters

For this tutorial, we will fine tune a pre-trained [Mask R-CNN](https://arxiv.org/abs/1703.06870) model in the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/). It contains 170 images with 345 instances of pedestrians, and we will use it  to train an instance segmentation model on this custom dataset.


You will use [Azure Machine Learning Pipelines](https://aka.ms/aml-pipelines) to define two pipeline steps: a data process step which split data into training and testing, and training step which trains and evaluates the model.  The trained model then registered to your AML workspace.


After the model is registered, you then deploy the model for testing using Azure Kubernetes Cluster (AKS).

This notebook  uses ASH storage and ASH cluster (ARC compute) for training, please make sure the following prerequisites are met.

## Prerequisite

*     A Kubernetes cluster deployed on Azure Stack Hub, connected to Azure through ARC.
     
   For details on how to deploy kubernetes cluster on Azure Stack Hub and enabling ARC connection to Azure, please follow [this guide](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/AML-ARC-Compute.md)
  

*     Datastore setup in Azure Machine Learning workspace backed up by Azure Stack Hub storage account.

   [This document](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md) is a detailed guide on how to create Azure Machine Learning workspace, create a  Azure Stack Hub Storage account, and setup datastore in AML workspace with ASH storage account.


*      Last but not least, you need to be able to run a Notebook. 

   If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration of notebooks located at [here](https://github.com/Azure/MachineLearningNotebooks) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc.

In [1]:
import os
from azureml.core import Workspace,Environment, Experiment, Datastore

from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.19.0 (c:\users\v-songshanli\anaconda3\envs\pythonproject\lib\site-packages), Requirement.parse('azureml-core~=1.18.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataset-runtime 1.19.0.post1 (c:\users\v-songshanli\anaconda3\envs\pythonproject\lib\site-packages), Requirement.parse('azureml-dataset-runtime~=1.18.0'), {'azureml-automl-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.19.0 (c:\users\v-songshanli\anaconda3\envs\pythonproject\lib\site-packages), Requirement.parse('azureml-core~=1.18.0')).
Failure while loading azureml_run_type_providers. Failed to load entr

### Create Workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [2]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


## Create or attach existing ArcKubernetesCompute

The attaching code here depends  python package azureml-contrib-k8s which current is in private preview. Install private preview branch of AzureML SDK by running following command (private preview):

<pre>
pip install --disable-pip-version-check --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-contrib-k8s-preview/D58E86006C65 azureml-contrib-k8s
</pre>

In [ ]:
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute

resource_id = "/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourceGroups/AML-stack-val/providers/Microsoft.Kubernetes/connectedClusters/kub-orlando-Test"

attach_name = "peymanarc"

attach_config = ArcKubernetesCompute.attach_configuration(resource_id=resource_id)

attach_result = ArcKubernetesCompute.attach(ws, attach_name, attach_config)

attach_result.wait_for_completion(show_output=True)

print(attach_result)

attach_name = "nc6"
compute_target = ws.compute_targets[attach_name]

## Register Dataset

After downloading and extracting the zip file from [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/) to your local machine, make sure you will have the following folder structure:

<pre>
PennFudanPed/
  PedMasks/
    FudanPed00001_mask.png
    FudanPed00002_mask.png
    FudanPed00003_mask.png
    FudanPed00004_mask.png
    ...
  PNGImages/
    FudanPed00001.png
    FudanPed00002.png
    FudanPed00003.png
    FudanPed00004.png
</pre>

Here PennFudanPed is a sub-folder directly under working folder of this note book.

Now you are ready to upload the data and register it as dataset in AML workspace in the following code (it takes about 1 min):

In [3]:
from azureml.core import Workspace, Dataset, Datastore

dataset_name = "pennfudan"
if dataset_name not  in ws.datasets:
    
    datastore_name = "ashstore"
    datastore =  Datastore.get(ws, datastore_name)
    
    src_dir, target_path = 'PennFudanPed', 'PennFudanPed'
    datastore.upload(src_dir, target_path, overwrite=True)

    # register data uploaded as AML dataset
    datastore_paths = [(datastore, target_path)]
    pd_ds = Dataset.File.from_files(path=datastore_paths)
    pd_ds.register(ws, dataset_name, "for Pedestrian Detection and Segmentation")

Uploading an estimated of 340 files
Uploading PennFudanPed\PedMasks\FudanPed00001_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00001_mask.png, 1 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00002_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00002_mask.png, 2 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00003_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00003_mask.png, 3 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00004_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00004_mask.png, 4 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00005_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00005_mask.png, 5 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00006_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00006_mask.png, 6 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00007_mask.png
Upl

Uploading PennFudanPed\PedMasks\FudanPed00058_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00058_mask.png, 61 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00059_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00059_mask.png, 62 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00061_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00061_mask.png, 63 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00062_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00062_mask.png, 64 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00065_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00065_mask.png, 65 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00066_mask.png
Uploaded PennFudanPed\PedMasks\FudanPed00066_mask.png, 66 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\FudanPed00067_mask.png
Uploaded PennFudanPed\PedMasks\Fu

Uploading PennFudanPed\PedMasks\PennPed00054_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00054_mask.png, 128 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\PennPed00055_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00055_mask.png, 129 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\PennPed00056_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00056_mask.png, 130 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\PennPed00057_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00057_mask.png, 131 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\PennPed00058_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00058_mask.png, 132 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\PennPed00059_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00059_mask.png, 133 files out of an estimated total of 340
Uploading PennFudanPed\PedMasks\PennPed00060_mask.png
Uploaded PennFudanPed\PedMasks\PennPed00

Uploading PennFudanPed\PNGImages\FudanPed00013.png
Uploaded PennFudanPed\PNGImages\FudanPed00013.png, 185 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00014.png
Uploaded PennFudanPed\PNGImages\FudanPed00014.png, 186 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00020.png
Uploaded PennFudanPed\PNGImages\FudanPed00020.png, 187 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00019.png
Uploaded PennFudanPed\PNGImages\FudanPed00019.png, 188 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00023.png
Uploaded PennFudanPed\PNGImages\FudanPed00023.png, 189 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00025.png
Uploaded PennFudanPed\PNGImages\FudanPed00025.png, 190 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00028.png
Uploaded PennFudanPed\PNGImages\FudanPed00028.png, 191 files out of an estimate

Uploading PennFudanPed\PNGImages\FudanPed00072.png
Uploaded PennFudanPed\PNGImages\FudanPed00072.png, 242 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00074.png
Uploaded PennFudanPed\PNGImages\FudanPed00074.png, 243 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\FudanPed00073.png
Uploaded PennFudanPed\PNGImages\FudanPed00073.png, 244 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00008.png
Uploaded PennFudanPed\PNGImages\PennPed00008.png, 245 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00001.png
Uploaded PennFudanPed\PNGImages\PennPed00001.png, 246 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00002.png
Uploaded PennFudanPed\PNGImages\PennPed00002.png, 247 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00003.png
Uploaded PennFudanPed\PNGImages\PennPed00003.png, 248 files out of an estimated total 

Uploading PennFudanPed\PNGImages\PennPed00055.png
Uploaded PennFudanPed\PNGImages\PennPed00055.png, 300 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00056.png
Uploaded PennFudanPed\PNGImages\PennPed00056.png, 301 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00058.png
Uploaded PennFudanPed\PNGImages\PennPed00058.png, 302 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00062.png
Uploaded PennFudanPed\PNGImages\PennPed00062.png, 303 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00063.png
Uploaded PennFudanPed\PNGImages\PennPed00063.png, 304 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00064.png
Uploaded PennFudanPed\PNGImages\PennPed00064.png, 305 files out of an estimated total of 340
Uploading PennFudanPed\PNGImages\PennPed00060.png
Uploaded PennFudanPed\PNGImages\PennPed00060.png, 306 files out of an estimated total of 340

## Create a Training-Test split data process Step

For this pipeline run, you will use two pipeline steps.  The first step is to split dataset into training and testing.

In [ ]:
# create run_config first
datastore_name = "ashstore"
datastore =  Datastore.get(ws, datastore_name)

env = Environment.from_dockerfile(
        name='pytorch-obj-seg',
        dockerfile='./aml_src/Dockerfile.gpu',
        conda_specification='./aml_src/conda-env.yaml')

aml_run_config = RunConfiguration()
aml_run_config.target = compute_target.name
aml_run_config.environment = env

source_directory = './aml_src'


# add a data process step

dataset = ws.datasets[dataset_name]

from azureml.data import OutputFileDatasetConfig

dest = (datastore, None)

train_split_data = OutputFileDatasetConfig(name="train_split_data", destination=dest).as_upload(overwrite=False)
test_split_data = OutputFileDatasetConfig(name="test_split_data", destination=dest).as_upload(overwrite=False)

split_step = PythonScriptStep(
    name="Train Test Split",
    script_name="obj_segment_step_data_process.py",
    arguments=["--data-path", dataset.as_named_input('pennfudan_data').as_mount(),
               "--train-split", train_split_data, "--test-split", test_split_data,
               "--test-size", 50],
    compute_target=compute_target,
    runconfig=aml_run_config,
    source_directory=source_directory,
    allow_reuse=False
)

## Create Training Step

In [ ]:
train_step = PythonScriptStep(
        name="training_step",
        script_name="obj_segment_step_training.py",
        arguments=[
            "--train-split", train_split_data.as_input(), "--test-split", test_split_data.as_input(),
            '--epochs', 60,  # 80
        ],

        compute_target=compute_target,
        runconfig=aml_run_config,
        source_directory=source_directory,
        allow_reuse=True
    )
    

## Create Experiment and Submit Pipeline Run

The split step takes about 8 mins. Training step takes about 25 mins per epoch for  vm comparable to Standard_DS3_v2

In [ ]:
experiment_name = 'obj_seg_step'
experiment = Experiment(workspace=ws, name=experiment_name)
pipeline_steps = [train_step]

pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

pipeline_run = experiment.submit(pipeline, regenerate_outputs=False)
pipeline_run.wait_for_completion()


## Register Model

The model saved at "'outputs/obj_segmentation.pkl'" is registered as   'obj_seg_model_aml'.  It contains both model parameters and network which are used by AML deployment and serving. 

In [ ]:
train_step_run = pipeline_run.find_step_run(train_step.name)[0]

model_name = 'obj_seg_model_aml_60' 
train_step_run.register_model(model_name=model_name, model_path='outputs/obj_segmentation.pkl')

## Test Registered Model

To test the trained model, you can create (or use existing) a AKS cluster for serving the model using AML deployment

### Create (or use existing) a AKS cluster for serving the model

In [ ]:
from azureml.core import Environment, Workspace, Model, ComputeTarget
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.compute_target import ComputeTargetException
from PIL import Image
from torchvision.transforms import functional as F
import numpy as np
import json

In [ ]:
ws = Workspace.from_config()

# Choose a name for your AKS cluster
aks_name = 'aks-service-2'

if aks_name not in  ws.compute_targets:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)
    is_new_compute  = True

    if aks_target.get_status() != "Succeeded":
        aks_target.wait_for_completion(show_output=True)
else:  
    aks_target =  ws.compute_targets[aks_name]   
    is_new_compute  = False
    
print("using compute target: ", aks_target.name)

### Deploy the model

In [ ]:
env = Environment.from_dockerfile(
        name='pytorch-obj-seg',
        dockerfile='./aml_src/Dockerfile.gpu',
        conda_specification='./aml_src/conda-env.yaml')

env.inferencing_stack_version='latest'

inference_config = InferenceConfig(entry_script='score.py', environment=env)
deploy_config = AksWebservice.deploy_configuration()

deployed_model = "obj_seg_model_aml" # model_name
model = ws.models[deployed_model]

service_name = 'objservice'

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deploy_config,
                       deployment_target=aks_target,
                       overwrite=True)

service.wait_for_deployment(show_output=True)


### Test the trained model Using the Deployed Service

In [ ]:
img_nums = ["00001"]
image_paths = ["PennFudanPed\\PNGImages\\FudanPed{}.png".format(item) for item in img_nums]
image_np_list = []
for image_path in image_paths:
    img = Image.open(image_path)
    img.show("input_image")
    img_rgb = img.convert("RGB")
    img_tensor = F.to_tensor(img_rgb)
    img_np = img_tensor.numpy()
    image_np_list.append(img_np.tolist())

inputs = json.dumps({"instances": image_np_list})
resp = service.run(inputs)
predicts = resp["predictions"]
p_str = json.dumps(predicts)

p_obj = json.loads(p_str)

for instance_pred in p_obj:
    image_data = instance_pred["masks"]
    img_np = np.array(image_data)
    output = Image.fromarray(img_np)
    output.show()

### Delete the newly created cluster

Note: This is important if you wish to avoid the cost of this cluster

In [ ]:
if is_new_compute:
    aks_target.delete()